---
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Cátedra de Investigación Operativa__\
__Autor: Rodrigo Maranzana__, Rmaranzana@frba.utn.edu.ar

---

In [1]:
import numpy as np
import pulp
from scipy.optimize import linprog

# Resolución FMC matricial con scipy.optimize.linprog

In [4]:
# Matriz nodo-arco.
A = np.array(
 [[ 1, 1, 0, 0, 0, 0, 0],
  [-1, 0, 1, 1, 0, 0, 0],
  [ 0,-1, 0, 0, 1, 0, 0],
  [ 0, 0,-1, 0, 0, 1, 0],
  [ 0, 0, 0, 0,-1, 0, 1],
  [ 0, 0, 0,-1, 0,-1,-1]]
)

# Vector de costos o distancias.
C = np.array([2, 2, 2, 5, 2, 1, 2])

# Vector de términos del lado derecho.
b = np.array([1, 0, 0, 0, 0, -1])

# print the parameters.
print(f"• Vector de costos: \n {C}\n")
print(f"• Matriz nodo-arco:\n {A}\n")
print(f"• Vector de término de lado derecho: \n {b}")

• Vector de costos: 
 [2 2 2 5 2 1 2]

• Matriz nodo-arco:
 [[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]]

• Vector de término de lado derecho: 
 [ 1  0  0  0  0 -1]


In [11]:
# Obtenemos cantidad de arcos:
dim_arcos = A.shape[1]

# Cargamos las cotas x>=0 como lista de tuplas, una para cada arco:
cotas = []

for arco_i in range(0, dim_arcos):
    
    cotas.append((0, np.inf))

In [10]:
# Resolvemos con scipy linprog:
res = linprog(C, A_eq=A, b_eq=b, bounds=cotas)

print(f'Arcos seleccionados: {res.x}')
print(f'Mínima distancia: {res.fun}')

Arcos seleccionados: [ 1. -0.  1.  0. -0.  1.  0.]
Mínima distancia: 5.0


# Resolución FMC algebraico con PuLP

In [23]:
import pulp

lp01 = pulp.LpProblem("ejercicio-inicial-fmc-shortest-path", pulp.LpMinimize)

# Arcos:
arcos = ['s2', 's3', '24', '2t', '35', '4t', '5t']

# Variables:
X = pulp.LpVariable.dicts('x', arcos, 0, None, cat='Continuous')

# Función objetivo:
lp01 += 2*X['s2'] + 2*X['s3'] + 2*X['24'] + 5*X['2t'] + 2*X['35'] + 1*X['4t'] + 2*X['5t'], "Z"

# # Restricciones:
lp01 +=  X['s2'] + X['s3'] == 1
lp01 +=  X['24'] + X['2t'] - X['s2'] == 0
lp01 +=  X['35'] - X['s3'] == 0
lp01 +=  X['4t'] - X['24'] == 0
lp01 +=  X['5t'] - X['35'] == 0
lp01 +=  -X['2t'] - X['4t'] - X['5t'] == -1

# Resolucion:
lp01.solve()

# Imprimimos el status del problema:
print(pulp.LpStatus[lp01.status])

# Imprimimos las variables en su valor óptimo:
for variable in lp01.variables():
    print("%s = %.2f" % (variable.name, variable.varValue))
    
# Imprimimos el funcional óptimo:
print(f'Función objetivo: {pulp.value(lp01.objective)}')

Optimal
x_24 = 1.00
x_2t = 0.00
x_35 = 0.00
x_4t = 1.00
x_5t = 0.00
x_s2 = 1.00
x_s3 = 0.00
Función objetivo: 5.0
